In [ ]:
import xarray as xr
import rioxarray as riox
import geopandas as gpd
import pandas as pd
from rasterio.plot import show
import os
import glob
import numpy as np
import panel as pn
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
from cartopy import crs
from xrspatial import convolution, focal, hillshade
import param as pm
from holoviews import streams
from tqdm.notebook import tqdm
from shapely.geometry import Polygon
from itertools import product
import matplotlib.pyplot as plt
import time
#pn.extension()
hv.notebook_extension('bokeh')
from dask.distributed import LocalCluster, Client
pn.param.ParamMethod.loading_indicator = True

In [ ]:
cluster = LocalCluster(n_workers=5, threads_per_worker=2)
cl = Client(cluster)
cl

In [ ]:
cper_f = '/mnt/d/CPER/data/vectors/Pasture_Boundaries/Shapefiles/cper_pdog_pastures_2017_clip.shp'

#rgb_f = '/mnt/e/202109/outputs/202109_29_30_RGB/CPER_202109_29_30_RGB_ortho.tif'
#ms_f = '/mnt/e/202109/outputs/202109_29_30_MS/CPER_202109_29_30_South_MS_ortho.tif'
#dsm_f = '/mnt/e/202109/outputs/202109_29_30_RGB/CPER_202109_29_30_RGB_DSM.tif'

#rgb_f = '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_ortho.tif'
#ms_f = '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_MS_ortho.tif'
#dsm_f = '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_DSM.tif'

img_f_dict = {
    '5W': {
        'group_1': {
            'rgb': '/mnt/e/202109/outputs/202109_5W_RGB/CPER_202109_5W_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_5W_MS/CPER_202109_5W_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_5W_RGB/CPER_202109_5W_RGB_dsm.tif'
        }
    },
    '29-30': {
        'group_1': {
            'rgb': '/mnt/e/202109/outputs/202109_29_30_RGB/CPER_202109_29_30_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_29_30_MS/CPER_202109_29_30_North_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_29_30_RGB/CPER_202109_29_30_RGB_DSM.tif'
        },
        'group_2': {
            'rgb': '/mnt/e/202109/outputs/202109_29_30_RGB/CPER_202109_29_30_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_29_30_MS/CPER_202109_29_30_South_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_29_30_RGB/CPER_202109_29_30_RGB_DSM.tif'
        }
    },
    '22W': {
        'group_1': {
            'rgb': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_DSM.tif'
        },
        'group_2': {
            'rgb': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_RGB_DSM.tif'
        }
    },
    '22E': {
        'group_1': {
            'rgb': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_DSM.tif'
        },
        'group_2': {
            'rgb': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_RGB_DSM.tif'
        },
        'group_3': {
            'rgb': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight3_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_22EW/CPER_202109_22EW_Flight3_RGB_DSM.tif'
        }
    },
    'CN': {
        'group_1': {
            'rgb': '/mnt/e/202109/outputs/202109_CN_RGB/Orthos/CPER_CN_Flight2_202109_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_CN_MS/CPER_202109_CN_Flight2_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_CN_RGB/DSMs/CPER_CN_Flight2_202109_RGB_DSM.tif'
        },
        'group_2': {
            'rgb': '/mnt/e/202109/outputs/202109_CN_RGB/Orthos/CPER_CN_Flight3_202109_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_CN_MS/CPER_202109_CN_Flight2_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_CN_RGB/DSMs/CPER_CN_Flight3_202109_RGB_DSM.tif'
        },
        'group_3': {
            'rgb': '/mnt/e/202109/outputs/202109_CN_RGB/Orthos/CPER_CN_Flight4_202109_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_CN_MS/CPER_202109_CN_Flight3_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_CN_RGB/DSMs/CPER_CN_Flight4_202109_RGB_DSM.tif'
        },
        'group_4': {
            'rgb': '/mnt/e/202109/outputs/202109_CN_RGB/Orthos/CPER_CN_Flight5_202109_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_CN_MS/CPER_202109_CN_Flight3_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_CN_RGB/DSMs/CPER_CN_Flight5_202109_RGB_DSM.tif'
        },
        'group_5': {
            'rgb': '/mnt/e/202109/outputs/202109_CN_RGB/Orthos/CPER_CN_Flight5_202109_RGB_ortho.tif',
            'ms': '/mnt/e/202109/outputs/202109_CN_MS/CPER_202109_CN_Flight4_MS_ortho.tif',
            'dsm': '/mnt/e/202109/outputs/202109_CN_RGB/DSMs/CPER_CN_Flight5_202109_RGB_DSM.tif'
        }
    }
}

cper_f = '/mnt/c/Users/TBGPEA-Sean/Desktop/Pdogs_UAS/cper_pdog_pastures_2017_clip.shp'

#rgb_f = '/mnt/c/Users/TBGPEA-Sean/Desktop/Pdogs_UAS/CPER_202109_5W_RGB_ortho.tif'
#ms_f = '/mnt/c/Users/TBGPEA-Sean/Desktop/Pdogs_UAS/CPER_202109_5W_MS_ortho.tif'
#dsm_f = '/mnt/c/Users/TBGPEA-Sean/Desktop/Pdogs_UAS/CPER_202109_5W_RGB_dsm.tif'

#ground_burrows_f = '/mnt/c/Users/TBGPEA-Sean/Desktop/Pdogs_UAS/ground/cper_pdog_points_2021Sept_burrows.shp'
#ground_other_f = '/mnt/c/Users/TBGPEA-Sean/Desktop/Pdogs_UAS/ground/cper_pdog_points_2021Sept_other.shp'
ground_polys_f = '/mnt/c/Users/TBGPEA-Sean/Desktop/Pdogs_UAS/ground/cper_pdog_polys_2021Sept.shp'
ground_polys_old_f = '/mnt/c/Users/TBGPEA-Sean/Desktop/Pdogs_UAS/ground/cper_pdog_polys_old_2021Sept.shp'
ground_polys_non_f = '/mnt/c/Users/TBGPEA-Sean/Desktop/Pdogs_UAS/ground/cper_pdog_polys_non_2021Sept.shp'

window_size_m = 30

In [ ]:
def hillshade(array, azimuth=225, angle_altitude=45):
    """https://github.com/rveciana/introduccion-python-geoespacial/blob/master/hillshade.py"""
    print(array.shape)
    azimuth = 360.0 - azimuth 
    x, y = np.gradient(array)
    slope = np.pi/2. - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    azimuthrad = azimuth*np.pi/180.
    altituderad = angle_altitude*np.pi/180.
    shaded = np.sin(altituderad)*np.sin(slope) + np.cos(altituderad)*np.cos(slope)*np.cos((azimuthrad - np.pi/2.) - aspect)
    return 255*(shaded + 1)/2


In [ ]:
from collections import namedtuple
from operator import mul

try:
    reduce = reduce
except NameError:
    from functools import reduce # py3k

Info = namedtuple('Info', 'start height')

def max_size(mat, value=0):
    """Find height, width of the largest rectangle containing all `value`'s.
    For each row solve "Largest Rectangle in a Histrogram" problem [1]:
    [1]: http://blog.csdn.net/arbuckle/archive/2006/05/06/710988.aspx
    """
    it = iter(mat)
    hist = [(el==value) for el in next(it, [])]
    max_size = max_rectangle_size(hist)
    for row in it:
        hist = [(1+h) if el == value else 0 for h, el in zip(hist, row)]
        max_size = max(max_size, max_rectangle_size(hist), key=area)
    return max_size

def max_rectangle_size(histogram):
    """Find height, width of the largest rectangle that fits entirely under
    the histogram.
    >>> f = max_rectangle_size
    >>> f([5,3,1])
    (3, 2)
    >>> f([1,3,5])
    (3, 2)
    >>> f([3,1,5])
    (5, 1)
    >>> f([4,8,3,2,0])
    (3, 3)
    >>> f([4,8,3,1,1,0])
    (3, 3)
    >>> f([1,2,1])
    (1, 3)
    Algorithm is "Linear search using a stack of incomplete subproblems" [1].
    [1]: http://blog.csdn.net/arbuckle/archive/2006/05/06/710988.aspx
    """
    stack = []
    top = lambda: stack[-1]
    max_size = (0, 0) # height, width of the largest rectangle
    pos = 0 # current position in the histogram
    for pos, height in enumerate(histogram):
        start = pos # position where rectangle starts
        while True:
            if not stack or height > top().height:
                stack.append(Info(start, height)) # push
            elif stack and height < top().height:
                max_size = max(max_size, (top().height, (pos - top().start)),
                               key=area)
                start, _ = stack.pop()
                continue
            break # height == top().height goes here

    pos += 1
    for start, height in stack:
        max_size = max(max_size, (height, (pos - start)), key=area)

    return max_size

def area(size):
    return reduce(mul, size)

In [ ]:
#pasture = '22E'
#group = 'group_3'
fig, ax = plt.subplots()
cper_gdf = gpd.read_file(cper_f)
hfig = display(cper_gdf.plot(ax=ax, color='none', edgecolor='black'), display_id=True, clear=True)

for pasture in tqdm(img_f_dict):
    print('\n\n----------\nPasture: ' + pasture)
    completed_csvs = [os.path.basename(x) for x in glob.glob('train_tiles/*.csv') if pasture in x]
    if len(img_f_dict[pasture]) == len(completed_csvs):
        print('Skipping pasture - tiling appears to be complete based on file list:')
        display(completed_csvs)
        continue
    else:
        for group in tqdm(img_f_dict[pasture]):
            print('---\nGroup: ' + group)

            # load in approapriate image data
            rgb_f = img_f_dict[pasture][group]['rgb']
            ms_f = img_f_dict[pasture][group]['ms']
            dsm_f = img_f_dict[pasture][group]['dsm']

            # load in ground data from shapefiles
            ground_polys_gdf = gpd.read_file(ground_polys_f)
            ground_polys_old_gdf = gpd.read_file(ground_polys_old_f)
            ground_polys_non_gdf = gpd.read_file(ground_polys_non_f)

            # filter ground data to pasture
            ground_polys_gdf = ground_polys_gdf[ground_polys_gdf['Pasture'] == pasture]
            ground_polys_old_gdf = ground_polys_old_gdf[ground_polys_old_gdf['Pasture'] == pasture]
            ground_polys_non_gdf = ground_polys_non_gdf[ground_polys_non_gdf['Pasture'] == pasture]

            # get initial bounding boxes of each ground sampling area
            ground_polys_bboxs_init = [pd.Series(data=[int(x) for x in row[1].geometry.bounds], 
                                             index=['minx', 'miny', 'maxx', 'maxy']) for row in ground_polys_gdf.iterrows()]
            ground_polys_old_bboxs_init = [pd.Series(data=[int(x) for x in row[1].geometry.bounds], 
                                             index=['minx', 'miny', 'maxx', 'maxy']) for row in ground_polys_old_gdf.iterrows()]
            ground_polys_non_bboxs_init = [pd.Series(data=[int(x) for x in row[1].geometry.bounds], 
                                             index=['minx', 'miny', 'maxx', 'maxy']) for row in ground_polys_non_gdf.iterrows()]
            
            # get comment field from sampling area
            ground_polys_cmts_init = ground_polys_gdf['Comment']
            ground_polys_old_cmts_init = ground_polys_old_gdf['Comment']
            ground_polys_non_cmts_init = ground_polys_non_gdf['Comment']
            
            # get the bounding box of the pasture
            past_bbox = cper_gdf[cper_gdf['Past_Name_'] == pasture].geometry.bounds.apply(lambda x: int(x))

            # open image data and mask and rename where appropriate
            rgb_xr = riox.open_rasterio(rgb_f, chunks={'y': 1000, 'x': 1000, 'band': 1}).sel(band=slice(0, 3))
            rgb_xr = rgb_xr.where(rgb_xr != 255)
            ms_xr = riox.open_rasterio(ms_f, chunks={'y': 500, 'x': 500, 'band': 1}).sel(band=[4, 3])
            ms_xr = ms_xr.where(ms_xr != 65535)
            dsm_xr = riox.open_rasterio(dsm_f, chunks={'y': 500, 'x': 500}).squeeze().drop('band')
            dsm_xr.name = 'DSM'
            dsm_xr = dsm_xr.where(dsm_xr > 0)

            # subset image data to pasture boundaries
            rgb_xr = rgb_xr.sel(y=slice(past_bbox['maxy'], past_bbox['miny']), 
                                x=slice(past_bbox['minx'], past_bbox['maxx']))

            ms_xr = ms_xr.sel(y=slice(past_bbox['maxy'], past_bbox['miny']), 
                                x=slice(past_bbox['minx'], past_bbox['maxx']))

            dsm_xr = dsm_xr.sel(y=slice(past_bbox['maxy'], past_bbox['miny']), 
                                x=slice(past_bbox['minx'], past_bbox['maxx']))

            # get count of any null data remaining in imagery within pasture boundaries
            ms_ct_null = ms_xr.isel(band=0).isnull().sum().compute()
            dsm_ct_null = dsm_xr.isnull().sum().compute()

            # if more than 1% of the multispectral data is null
            # get the largest rectangle (to nearest 10 m) of non-null multispectral data
            if (ms_ct_null/(ms_xr.shape[1]*ms_xr.shape[2])) > 0.01:
                # coarsen imagery to approximately 10 m
                ms_1m_coarse_val = int(10.0/ms_xr.rio.resolution()[0])
                ms_1m_res = ms_1m_coarse_val * ms_xr.rio.resolution()[0]
                ms_1m = ms_xr.isel(band=1).notnull().astype('int').coarsen(x=ms_1m_coarse_val,
                                                                          y=ms_1m_coarse_val, boundary='trim').max().compute()

                # get the size of the largest rectangle with no null values
                cln_rect = max_size(ms_1m.values, value=1)
                cln_rect

                # get the number of rows and columns to iterate through to find lower-left coords of non-null rectangle
                x_chk_n = (ms_1m.x.max() - (ms_1m.x.min() + ((cln_rect[1] - 1) * ms_1m_res))) / ms_1m_res + 1
                y_chk_n = (ms_1m.y.max() - (ms_1m.y.min() + ((cln_rect[0] - 1) * ms_1m_res))) / ms_1m_res + 1

                # iterate through the rows and columns and save all starting coordinates with non-null rectangles
                x_cln_list = []
                y_cln_list = []
                for x in tqdm(np.arange(ms_1m.x.min(), ms_1m.x.min() + x_chk_n * ms_1m_res, ms_1m_res)):
                    for y in np.arange(ms_1m.y.min(), ms_1m.y.min() + y_chk_n * ms_1m_res, ms_1m_res):
                        chk_null = (ms_1m.sel(x=slice(x, x + cln_rect[1] * ms_1m_res-1),
                                             y=slice(y + cln_rect[0] * ms_1m_res-1, y)) == 1).all()
                        if chk_null:
                            x_cln_list.append(x)
                            y_cln_list.append(y)


                # save the minimum starting coordinates
                coords_cln = pd.Series({'x': x_cln_list[np.argmin(y_cln_list)], 'y': np.min(y_cln_list)})
                coords_cln

                # update the extent of the multispectral image and rechunk for NDVI calc later
                ms_xr = ms_xr.sel(y=slice(coords_cln['y'] + cln_rect[0]*ms_1m_res, coords_cln['y']), 
                                  x=slice(coords_cln['x'], coords_cln['x'] + cln_rect[1]*ms_1m_res)).chunk({'y': 500, 'x': 500, 'band': -1})

            # if more than 1% of the RGB DSM data is null
            # get the largest rectangle (to nearest 10 m) of non-null RGB DSM data
            if (dsm_ct_null/(dsm_xr.shape[0]*dsm_xr.shape[1])) > 0.01:
                # coarsen imagery to approximately 10 m
                dsm_1m_coarse_val = int(10.0/dsm_xr.rio.resolution()[0])
                dsm_1m_res = dsm_1m_coarse_val * dsm_xr.rio.resolution()[0]
                dsm_1m = dsm_xr.notnull().astype('int').coarsen(x=dsm_1m_coarse_val,
                                                                y=dsm_1m_coarse_val, boundary='trim').max().compute()

                # get the size of the largest rectangle with no null values
                cln_rect = max_size(dsm_1m.values, value=1)
                cln_rect

                # get the number of rows and columns to iterate through to find lower-left coords of non-null rectangle
                x_chk_n = (dsm_1m.x.max() - (dsm_1m.x.min() + ((cln_rect[1] - 1) * dsm_1m_res))) / dsm_1m_res + 1
                y_chk_n = (dsm_1m.y.max() - (dsm_1m.y.min() + ((cln_rect[0] - 1) * dsm_1m_res))) / dsm_1m_res + 1

                # iterate through the rows and columns and save all starting coordinates with non-null rectangles
                x_cln_list = []
                y_cln_list = []
                for x in tqdm(np.arange(dsm_1m.x.min(), dsm_1m.x.min() + x_chk_n * dsm_1m_res, dsm_1m_res)):
                    for y in np.arange(dsm_1m.y.min(), dsm_1m.y.min() + y_chk_n * dsm_1m_res, dsm_1m_res):
                        chk_null = (dsm_1m.sel(x=slice(x, x + cln_rect[1] * dsm_1m_res-1),
                                             y=slice(y + cln_rect[0] * dsm_1m_res-1, y)) == 1).all()
                        if chk_null:
                            x_cln_list.append(x)
                            y_cln_list.append(y)


                # save the minimum starting coordinates
                coords_cln = pd.Series({'x': x_cln_list[np.argmin(y_cln_list)], 'y': np.min(y_cln_list)})
                coords_cln

                # update the extent of the RGB and DSM images
                rgb_xr = rgb_xr.sel(y=slice(coords_cln['y'] + cln_rect[0]*dsm_1m_res, coords_cln['y']), 
                                  x=slice(coords_cln['x'], coords_cln['x'] + cln_rect[1]*dsm_1m_res))
                dsm_xr = dsm_xr.sel(y=slice(coords_cln['y'] + cln_rect[0]*dsm_1m_res, coords_cln['y']), 
                                  x=slice(coords_cln['x'], coords_cln['x'] + cln_rect[1]*dsm_1m_res))

            # get the minimum bounding box of all non-null data
            past_bbox['minx'] = max(rgb_xr.x.min(), ms_xr.x.min(), dsm_xr.x.min(), past_bbox['minx'])
            past_bbox['miny'] = max(rgb_xr.y.min(), ms_xr.y.min(), dsm_xr.y.min(), past_bbox['miny'])
            past_bbox['maxx'] = min(rgb_xr.x.max(), ms_xr.x.max(), dsm_xr.x.max(), past_bbox['maxx'])
            past_bbox['maxy'] = min(rgb_xr.y.max(), ms_xr.y.max(), dsm_xr.y.max(), past_bbox['maxy'])

            # buffer the pasture bounding box by the window size to ensure tiles are completely within pastures
            past_bbox_buff = past_bbox.copy(deep=True)
            past_bbox_buff[['minx', 'miny']] = past_bbox[['minx', 'miny']].apply(lambda x: x + window_size_m)
            past_bbox_buff[['maxx', 'maxy']] = past_bbox[['maxx', 'maxy']].apply(lambda x: x - window_size_m)
            num_windows = round(((past_bbox_buff['maxx'] - past_bbox_buff['minx']) * (past_bbox_buff['maxy'] - past_bbox_buff['miny']))/10000/10)

            # plot the current pasture and bounding box of the analsis area in the output preview
            cper_gdf[cper_gdf['Past_Name_'] == pasture].plot(ax=ax)
            gpd.GeoSeries(Polygon([(past_bbox['minx'], past_bbox['miny']),
                     (past_bbox['minx'], past_bbox['maxy']),
                     (past_bbox['maxx'], past_bbox['maxy']),
                     (past_bbox['maxx'], past_bbox['miny'])])).plot(ax=ax, edgecolor='red', color='none')
            fig.canvas.draw()
            hfig.update(fig)

            # subset all image data to the bounding box
            rgb_xr = rgb_xr.sel(y=slice(past_bbox['maxy'], past_bbox['miny']), 
                                x=slice(past_bbox['minx'], past_bbox['maxx']))

            ms_xr = ms_xr.sel(y=slice(past_bbox['maxy'], past_bbox['miny']), 
                                x=slice(past_bbox['minx'], past_bbox['maxx']))

            dsm_xr = dsm_xr.sel(y=slice(past_bbox['maxy'], past_bbox['miny']), 
                                x=slice(past_bbox['minx'], past_bbox['maxx']))

            # calculate NDVI
            ndvi_xr = (ms_xr.sel(band=4).astype('float') - ms_xr.sel(band=3).astype('float'))\
            / (ms_xr.sel(band=4).astype('float') + ms_xr.sel(band=3).astype('float'))

            # get cell size of the the DSM in prep foro TPI calculation
            cellsize_x, cellsize_y = convolution.calc_cellsize(dsm_xr)

            # prepare an annulus kernel with a ring at a distance from 5-10 cells away from focal point
            outer_radius = str(cellsize_x * 10) + "m"
            inner_radius = str(cellsize_x * 5) + "m"
            kernel = convolution.annulus_kernel(cellsize_x, cellsize_y, outer_radius, inner_radius)

            # create the TPI image
            tpi_xr = dsm_xr - focal.apply(dsm_xr, kernel)

            # create a hillshade image
            shade_xr = xr.apply_ufunc(hillshade, dsm_xr, vectorize=False, dask='parallelized', output_dtypes='int')
            shade_xr.name = 'shade'

            # get any ground sampling areas that lie within the anlysis bounding box
            ground_polys_bboxs = []
            ground_polys_cmnts = []
            coord_starts_polys = []
            for idx, i in enumerate(ground_polys_bboxs_init):
                if not any([i['minx'] < past_bbox['minx'],
                        i['maxx'] > past_bbox['maxx'],
                        i['miny'] < past_bbox['miny'],
                        i['maxy'] > past_bbox['maxy']]):
                    ground_polys_bboxs.append(i)
                    # get lower-left starting coordinates
                    y_starts_tmp = np.arange(i['miny'], i['maxy'], window_size_m)
                    x_starts_tmp = np.arange(i['minx'], i['maxx'], window_size_m)
                    ground_polys_cmnts = ground_polys_cmnts + list(np.repeat(ground_polys_cmts_init.iloc[idx], 
                                                        len(y_starts_tmp)*len(x_starts_tmp)))
                    coord_starts_polys = coord_starts_polys + list(product(x_starts_tmp, y_starts_tmp))
            coord_starts_polys = np.array(coord_starts_polys)


            ground_polys_non_bboxs = []
            ground_polys_non_cmnts = []
            coord_starts_polys_non = []
            for idx, i in enumerate(ground_polys_non_bboxs_init):
                if not any([i['minx'] < past_bbox['minx'],
                        i['maxx'] > past_bbox['maxx'],
                        i['miny'] < past_bbox['miny'],
                        i['maxy'] > past_bbox['maxy']]):
                    ground_polys_non_bboxs.append(i)
                    y_starts_tmp_non = np.arange(i['miny'], i['maxy'], window_size_m)
                    x_starts_tmp_non = np.arange(i['minx'], i['maxx'], window_size_m)
                    ground_polys_non_cmnts = ground_polys_non_cmnts + list(np.repeat(ground_polys_non_cmts_init.iloc[idx],
                                                       len(y_starts_tmp_non)*len(x_starts_tmp_non)))
                    coord_starts_polys_non = coord_starts_polys_non + list(product(x_starts_tmp_non, y_starts_tmp_non))
            coord_starts_polys_non = np.array(coord_starts_polys_non)
            
            ground_polys_old_bboxs = []
            ground_polys_old_cmnts = []
            coord_starts_polys_old = []
            for idx, i in enumerate(ground_polys_old_bboxs_init):
                if not any([i['minx'] < past_bbox['minx'],
                        i['maxx'] > past_bbox['maxx'],
                        i['miny'] < past_bbox['miny'],
                        i['maxy'] > past_bbox['maxy']]):
                    ground_polys_old_bboxs.append(i)
                    y_starts_tmp_old = np.arange(i['miny'], i['maxy'], window_size_m)
                    x_starts_tmp_old = np.arange(i['minx'], i['maxx'], window_size_m)
                    ground_polys_old_cmnts = ground_polys_old_cmnts + list(np.repeat(ground_polys_old_cmts_init.iloc[idx],
                                                       len(y_starts_tmp_old)*len(x_starts_tmp_old)))
                    coord_starts_polys_old = coord_starts_polys_old + list(product(x_starts_tmp_old, y_starts_tmp_old))
            coord_starts_polys_old = np.array(coord_starts_polys_old)

            # get a random set of lower-left starting coordinates for producing tiles
            y_starts_past = np.arange(past_bbox_buff['miny'], past_bbox_buff['maxy'], window_size_m)
            x_starts_past = np.arange(past_bbox_buff['minx'], past_bbox_buff['maxx'], window_size_m)
            coord_starts_past = list(product(x_starts_past, y_starts_past))
            np.random.seed(4321)
            coord_starts_past_sub = np.array(coord_starts_past)[list(np.random.choice(np.array(coord_starts_past).shape[0],
                                                                                      size=num_windows))]
   

            # create dataframes of all bounding boxes for output
            df_bboxes_past = pd.DataFrame({
                'Pasture': pasture,
                'Tile': ['random_' + str(i) for i in range(coord_starts_past_sub.shape[0])],
                'min_x': [i[0] for i in coord_starts_past_sub],
                'min_y': [i[1] for i in coord_starts_past_sub],
                'Type': 'random',
                'Train': 1,
                'Digitize': 1,
                'Poly_ID': 'None'
            })
            df_bboxes_past.loc[df_bboxes_past.sample(frac=0.30, axis=0, random_state=4321).index, 'Train'] = 0
            df_bboxes_polys = pd.DataFrame({
                'Pasture': pasture,
                'Tile': ['burrows_active_' + str(i) for i in range(coord_starts_polys.shape[0])],
                'min_x': [i[0] for i in coord_starts_polys],
                'min_y': [i[1] for i in coord_starts_polys],
                'Type': 'burrows_active',
                'Train': 0,
                'Digitize': 1,
                'Poly_ID': ground_polys_cmnts
            })
            df_bboxes_polys.loc[df_bboxes_polys.sample(frac=0.50, axis=0, random_state=4321).index, 'Train'] = 0
            df_bboxes_polys.loc[df_bboxes_polys.sample(frac=0.50, axis=0, random_state=4321).index, 'Digitize'] = 0
            df_bboxes_polys_old = pd.DataFrame({
                'Pasture': pasture,
                'Tile': ['burrows_old_' + str(i) for i in range(coord_starts_polys_old.shape[0])],
                'min_x': [i[0] for i in coord_starts_polys_old],
                'min_y': [i[1] for i in coord_starts_polys_old],
                'Type': 'burrows_old',
                'Train': 0,
                'Digitize': 0,
                'Poly_ID': ground_polys_old_cmnts
            })
            df_bboxes_polys_non = pd.DataFrame({
                'Pasture': pasture,
                'Tile': ['non_burrows_' + str(i) for i in range(coord_starts_polys_non.shape[0])],
                'min_x': [i[0] for i in coord_starts_polys_non],
                'min_y': [i[1] for i in coord_starts_polys_non],
                'Type': 'non_burrows',
                'Train': 0,
                'Digitize': 0,
                'Poly_ID': ground_polys_non_cmnts
            })

            # combine dataframes of individual ground sampling types
            df_bboxes = pd.concat([df_bboxes_past, 
                                   df_bboxes_polys,
                                   df_bboxes_polys_old,
                                   df_bboxes_polys_non]).reset_index(drop=True).reset_index().rename(columns={'index': 'ID'})

            # create unique ID's for each tile
            df_bboxes['ID'] = df_bboxes.apply(lambda x: x.Pasture + '_' + str(x.ID), axis=1)

            # calculate the upper-right coordinates of each tile
            df_bboxes['max_x'] = df_bboxes['min_x'] + window_size_m
            df_bboxes['max_y'] = df_bboxes['min_y'] + window_size_m

            # label each tile with the image group used to produce it
            df_bboxes['img_group'] = group

            print('Calculating the following number of tiles by type:' + '\n--------')
            display(df_bboxes.groupby('Type')['ID'].count())

            # check if an output already exisit for the pasture
            # if so, append the current tile list dataframe to the existing one after updating the ID numbers
            if os.path.exists('train_tiles/train_bboxes_' + pasture + '.csv'):
                print('Tiles already exist for this pasture. Updating tile numbers accordingly.')
                df_bboxes_exist = pd.read_csv('train_tiles/train_bboxes_' + pasture + '.csv')
                df_bboxes = df_bboxes[(df_bboxes['Poly_ID'] == 'None') |
                                      ~(df_bboxes['Poly_ID'].isin(df_bboxes_exist['Poly_ID'].unique()))]
                for t, d in df_bboxes_exist.groupby('Type'):
                    t_ct = d['Tile'].apply(lambda x: int(x.split('_')[-1])).max() + 1
                    df_bboxes.loc[df_bboxes['Type'] == t, 
                              'Tile'] = df_bboxes.loc[df_bboxes['Type'] == t, 
                                                      'Tile'].apply(lambda x: '_'.join(['_'.join(x.split('_')[:-1]), 
                                                                                        str(int(x.split('_')[-1]) + t_ct)]))
                id_ct = df_bboxes_exist['ID'].apply(lambda x: int(x.split('_')[-1])).max() + 1
                df_bboxes.loc[:, 'ID'] = df_bboxes['ID'].apply(lambda x: '_'.join([x.split('_')[0], 
                                                                                   str(int(x.split('_')[-1]) + id_ct)]))
                df_bboxes_exist.to_csv('train_tiles/train_bboxes_' + pasture + '_' + group + '.csv', index=False)
                pd.concat([df_bboxes_exist, df_bboxes]).to_csv('train_tiles/train_bboxes_' + pasture + '.csv', index=False)
            else:
                df_bboxes.to_csv('train_tiles/train_bboxes_' + pasture + '.csv', index=False)

            # make directories for each tile type for the given pasture, if they don't already exist
            for t in df_bboxes['Type'].unique():
                if not os.path.exists('train_tiles/' + pasture + '_' + t):
                    os.mkdir('train_tiles/' + pasture + '_' + t)

            # create tiles and write to disk
            if True:
                for row in tqdm(df_bboxes.iterrows(), total=df_bboxes.shape[0]):
                    s_tmp = row[1]
                    rgb_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
                                x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).rio.to_raster(
                        'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_rgb.tif')

                    ndvi_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
                                    x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).rio.to_raster(
                        'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_ndvi.tif')

                    shade_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
                                    x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).astype('float32').rio.to_raster(
                        'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_shade.tif')

                    tpi_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
                                    x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).rio.to_raster(
                        'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_tpi.tif')

                    dsm_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
                                    x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).rio.to_raster(
                        'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_dsm.tif')


In [ ]:
"""df_bboxes_polys = pd.DataFrame({
    'Pasture': pasture,
    'Tile': ['burrows_active_' + str(i) for i in range(coord_starts_polys.shape[0])],
    'min_x': [i[0] for i in coord_starts_polys],
    'min_y': [i[1] for i in coord_starts_polys],
    'Type': 'burrows_active',
    'Train': 0,
    'Digitize': 1,
    'Poly_ID': ground_polys_cmnts
})"""

#len(ground_polys_cmnts)
coord_starts_polys.shape[0]

In [ ]:
len(ground_polys_bboxs_init)

In [ ]:
ground_polys_bboxs = []
ground_polys_cmnts = []
coord_starts_polys = []
y_starts_polys = np.array([])
x_starts_polys = np.array([])
for idx, i in enumerate(ground_polys_bboxs_init):
    if not any([i['minx'] < past_bbox['minx'],
            i['maxx'] > past_bbox['maxx'],
            i['miny'] < past_bbox['miny'],
            i['maxy'] > past_bbox['maxy']]):
        ground_polys_bboxs.append(i)
        # get lower-left starting coordinates
        y_starts_tmp = np.arange(i['miny'], i['maxy'], window_size_m)
        x_starts_tmp = np.arange(i['minx'], i['maxx'], window_size_m)
        y_starts_polys = np.append(y_starts_polys, y_starts_tmp)
        x_starts_polys = np.append(x_starts_polys, x_starts_tmp)
        print(len(y_starts_tmp),len(x_starts_tmp))
        print(i)
        ground_polys_cmnts = ground_polys_cmnts + list(np.repeat(ground_polys_cmts_init.iloc[idx], 
                                            len(y_starts_tmp)*len(x_starts_tmp)))                                             
        coord_starts_polys = coord_starts_polys + list(product(x_starts_tmp, y_starts_tmp))
coord_starts_polys = np.array(coord_starts_polys)

In [ ]:
print(y_starts_polys, x_starts_polys)

In [ ]:
coord_starts_polys

In [ ]:

# get any ground sampling areas that lie within the anlysis bounding box
ground_polys_bboxs = []
ground_polys_cmnts = []
y_starts_polys = np.array([])
x_starts_polys = np.array([])
for idx, i in enumerate(ground_polys_bboxs_init):
    if not any([i['minx'] < past_bbox['minx'],
            i['maxx'] > past_bbox['maxx'],
            i['miny'] < past_bbox['miny'],
            i['maxy'] > past_bbox['maxy']]):
        ground_polys_bboxs.append(i)
        # get lower-left starting coordinates
        y_starts_tmp = np.arange(i['miny'], i['maxy'], window_size_m)
        x_starts_tmp = np.arange(i['minx'], i['maxx'], window_size_m)
        y_starts_polys = np.append(y_starts_polys, y_starts_tmp)
        x_starts_polys = np.append(x_starts_polys, x_starts_tmp)
        ground_polys_cmnts = ground_polys_cmnts + list(np.repeat(ground_polys_cmts_init.iloc[idx], 
                                            len(y_starts_tmp)*len(x_starts_tmp)*2))                                             
coord_starts_polys = np.array(list(product(x_starts_polys, y_starts_polys)))


ground_polys_non_bboxs = []
ground_polys_non_cmnts = []
y_starts_polys_non = np.array([])
x_starts_polys_non = np.array([])
for idx, i in enumerate(ground_polys_non_bboxs_init):
    if not any([i['minx'] < past_bbox['minx'],
            i['maxx'] > past_bbox['maxx'],
            i['miny'] < past_bbox['miny'],
            i['maxy'] > past_bbox['maxy']]):
        ground_polys_non_bboxs.append(i)
        y_starts_tmp_non = np.arange(i['miny'], i['maxy'], window_size_m)
        x_starts_tmp_non = np.arange(i['minx'], i['maxx'], window_size_m)
        y_starts_polys_non = np.append(y_starts_polys_non, y_starts_tmp_non)
        x_starts_polys_non = np.append(x_starts_polys_non, x_starts_tmp_non)
        ground_polys_non_cmnts.append(np.repeat(ground_polys_non_cmts_init.iloc[idx],
                                           len(y_starts_tmp_non)*len(x_starts_tmp_non)*2))
coord_starts_polys_non = np.array(list(product(x_starts_polys_non, y_starts_polys_non)))

ground_polys_old_bboxs = []
ground_polys_old_cmnts = []
y_starts_polys_old = np.array([])
x_starts_polys_old = np.array([])
for idx, i in enumerate(ground_polys_old_bboxs_init):
    if not any([i['minx'] < past_bbox['minx'],
            i['maxx'] > past_bbox['maxx'],
            i['miny'] < past_bbox['miny'],
            i['maxy'] > past_bbox['maxy']]):
        ground_polys_old_bboxs.append(i)
        y_starts_tmp_old = np.arange(i['miny'], i['maxy'], window_size_m)
        x_starts_tmp_old = np.arange(i['minx'], i['maxx'], window_size_m)
        y_starts_polys_old = np.append(y_starts_polys_old, y_starts_tmp_old)
        x_starts_polys_old = np.append(x_starts_polys_old, x_starts_tmp_old)
        ground_polys_old_cmnts.append(np.repeat(ground_polys_old_cmts_init.iloc[idx],
                                           len(y_starts_tmp_old)*len(x_starts_tmp_olt)*2))
coord_starts_polys_old = np.array(list(product(x_starts_polys_old, y_starts_polys_old)))

# get a random set of lower-left starting coordinates for producing tiles
y_starts_past = np.arange(past_bbox_buff['miny'], past_bbox_buff['maxy'], window_size_m)
x_starts_past = np.arange(past_bbox_buff['minx'], past_bbox_buff['maxx'], window_size_m)
coord_starts_past = list(product(x_starts_past, y_starts_past))
np.random.seed(4321)
coord_starts_past_sub = np.array(coord_starts_past)[list(np.random.choice(np.array(coord_starts_past).shape[0],
                                                                          size=num_windows))]


In [ ]:
#coord_starts_polys
ground_polys_cmnts

In [ ]:
# create dataframes of all bounding boxes for output
df_bboxes_past = pd.DataFrame({
    'Pasture': pasture,
    'Tile': ['random_' + str(i) for i in range(coord_starts_past_sub.shape[0])],
    'min_x': [i[0] for i in coord_starts_past_sub],
    'min_y': [i[1] for i in coord_starts_past_sub],
    'Type': 'random',
    'Train': 1,
    'Digitize': 1
})
df_bboxes_past.loc[df_bboxes_past.sample(frac=0.30, axis=0, random_state=4321).index, 'Train'] = 0
df_bboxes_polys = pd.DataFrame({
    'Pasture': pasture,
    'Tile': ['burrows_active_' + str(i) for i in range(coord_starts_polys.shape[0])],
    'min_x': [i[0] for i in coord_starts_polys],
    'min_y': [i[1] for i in coord_starts_polys],
    'Type': 'burrows_active',
    'Train': 0,
    'Digitize': 1,
    'Poly_ID': ground_polys_cmnts
})
df_bboxes_polys.loc[df_bboxes_polys.sample(frac=0.50, axis=0, random_state=4321).index, 'Train'] = 0
df_bboxes_polys.loc[df_bboxes_polys.sample(frac=0.50, axis=0, random_state=4321).index, 'Digitize'] = 0
df_bboxes_polys_old = pd.DataFrame({
    'Pasture': pasture,
    'Tile': ['burrows_old_' + str(i) for i in range(coord_starts_polys_old.shape[0])],
    'min_x': [i[0] for i in coord_starts_polys_old],
    'min_y': [i[1] for i in coord_starts_polys_old],
    'Type': 'burrows_old',
    'Train': 0,
    'Digitize': 0,
    'Poly_ID': ground_polys_old_cmnts
})
df_bboxes_polys_non = pd.DataFrame({
    'Pasture': pasture,
    'Tile': ['non_burrows_' + str(i) for i in range(coord_starts_polys_non.shape[0])],
    'min_x': [i[0] for i in coord_starts_polys_non],
    'min_y': [i[1] for i in coord_starts_polys_non],
    'Type': 'non_burrows',
    'Train': 0,
    'Digitize': 0,
    'Poly_ID': ground_polys_non_cmnts
})

In [ ]:
df_bboxes_polys['Poly_ID'].iloc[0] = 'None'
df_bboxes_polys

In [ ]:
df_bboxes_polys[(df_bboxes_polys['Poly_ID'] == 'None') |
                                      ~(df_bboxes_polys['Poly_ID'].isin(['cn1', 'None']))]

In [ ]:
df_bboxes_polys = pd.DataFrame({
    'Pasture': pasture,
    'Tile': ['burrows_active_' + str(i) for i in range(coord_starts_polys.shape[0])],
    'min_x': [i[0] for i in coord_starts_polys],
    'min_y': [i[1] for i in coord_starts_polys],
    'Type': 'burrows_active',
    'Train': 0,
    'Digitize': 1,
    'Poly_ID': ground_polys_cmnts
})

In [ ]:
[i[0] for i in coord_starts_polys]

In [ ]:
df_bboxes.loc[df_bboxes['Type'] == t, 
                                          'Tile'].apply(lambda x: '_'.join(['_'.join(x.split('_')[:-1]), 
                                                                            str(int(x.split('_')[-1]) + t_ct)]))

In [ ]:
df_bboxes['Tile'] = 'burrows_active_11'

In [ ]:
rgb_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
            x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).rio.to_raster(
    'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_rgb.tif')

ndvi_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
                x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).rio.to_raster(
    'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_ndvi.tif')

shade_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
                x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).astype('float32').rio.to_raster(
    'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_shade.tif')

tpi_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
                x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).rio.to_raster(
    'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_tpi.tif')

dsm_xr.sel(y=slice(s_tmp['min_y'] + window_size_m*2.0, s_tmp['min_y'] - window_size_m), 
                x=slice(s_tmp['min_x'] - window_size_m, s_tmp['min_x'] + window_size_m*2.0)).rio.to_raster(
    'train_tiles/' + pasture + '_' + s_tmp['Type'] + '/' + pasture + '_' + s_tmp['Tile'] + '_dsm.tif')

In [ ]:
class TrainApp(pm.Parameterized):
       
    pasture = '5W'
    window_size_m = 15
    num_windows = 24

    select_tile = pm.Selector(objects=['tile ' + str(i+1) for i in range(num_windows)], default='tile 1')
    
    map_opts = dict(projection=crs.UTM(13), responsive=False, xaxis=None, yaxis=None, width=500, height=500,
                     padding=0, tools=['pan', 'box_zoom'], 
                     active_tools=['wheel_zoom'], toolbar='left')
    
    map_args = dict(crs=crs.UTM(13), rasterize=False, project=False, dynamic=True)
    
    draw_poly_opts = dict(fill_color=['red'], fill_alpha=[0.10], line_color=['red'],
             line_width=[2])  
    
    saved_poly_opts = dict(fill_color='yellow', fill_alpha=0.20, line_color='yellow', line_width=2)
    
    rgb_xr_past = rgb_xr
    ndvi_xr_past = ndvi_xr
    shade_xr_past = shade_xr
    tpi_xr_past = tpi_xr
    
    burrow_edit_list = [[] for x in range(num_windows)]
    
    burrow_poly_list = [[] for x in range(num_windows)]
    
    #burrows = hv.Polygons(burrow_poly_list[0]).opts(line_color='green')
    
    #burrow_stream = streams.PolyDraw(source=burrows, drag=True,
    #                                 show_vertices=False, styles=poly_opts)
    #burrow_edit_stream = streams.PolyEdit(source=burrows, shared=True)
    
    #btn_save = pn.widgets.Button(name='Save polygons', button_type='primary')
    
    save_action = pm.Action(lambda x: x.param.trigger('save_action'), label='Save polygons')
    
    ndvi_range = pn.widgets.RangeSlider(start=0.05, end=0.25, step=0.01)
    
    _ndvi_range = ndvi_range.param
    #test = 0
    
    def __init__(self, **params):
        super(TrainApp, self).__init__(**params)
    
    @pm.depends('save_action', watch=True)
    def update_burrows(self):
        #self.burrow_edit_list[self.i] = self.burrow_stream.element
        
        if len(self.burrow_poly_list[self.i]) > 0 & len(self.burrow_stream.element.data) > 0:
            tile_burrows_tmp = gpd.GeoDataFrame(data=self.burrow_edit_list[self.i].data)
            tile_burrows_tmp.set_geometry(tile_burrows_tmp.apply(lambda row: Polygon(zip(row['x'], row['y'])), axis=1), inplace=True)
            tile_burrows_tmp.set_crs(epsg='32613', inplace=True)
            tile_burrows_tmp = tile_burrows_tmp.drop(columns=['x', 'y'])
            self.burrow_poly_list[self.i] = self.burrow_poly_list[self.i].append(tile_burrows_tmp)
        elif len(self.burrow_stream.element.data) > 0:
            tile_burrows_tmp = gpd.GeoDataFrame(data=self.burrow_stream.element.data)
            tile_burrows_tmp.set_geometry(tile_burrows_tmp.apply(lambda row: Polygon(zip(row['x'], row['y'])), axis=1), inplace=True)
            tile_burrows_tmp.set_crs(epsg='32613', inplace=True)
            tile_burrows_tmp = tile_burrows_tmp.drop(columns=['x', 'y'])
            self.burrow_poly_list[self.i] = tile_burrows_tmp
        #self.burrow_poly_list[self.i][0]['x'] = self.burrow_poly_list[self.i][0]['x'].astype('int')
        #self.burrow_poly_list[self.i][0]['y'] = self.burrow_poly_list[self.i][0]['y'].astype('int')
        #self.burrow_poly_list[self.i]['line_color'] = ['green' for x in self.burrow_poly_list[0]['line_color']]
    
        tile_burrows = self.burrow_poly_list[self.i]

        _burrows = tile_burrows.hvplot(**self.saved_poly_opts)
        
        self.burrow_stream = streams.PolyDraw(source=_burrows, drag=True,
                                   show_vertices=False, styles=self.draw_poly_opts)

        self.burrow_edit_stream = streams.PolyEdit(source=_burrows, shared=True)
        
        return _burrows
    
    @pm.depends('select_tile', 'save_action')
    def load_image(self):
        self.i = int(self.select_tile.split(' ')[-1])
        #self.burrow_poly_list[self.i] = self.burrow_edit_stream.data
        rgb_sub = self.rgb_xr_past.sel(y=slice(y_starts_sub[self.i] + window_size_m * 2.0, 
                             y_starts_sub[self.i] - window_size_m), 
                     x=slice(x_starts_sub[self.i] - window_size_m,
                             x_starts_sub[self.i] + window_size_m * 2.0)).persist()
        ndvi_sub = self.ndvi_xr_past.sel(y=slice(y_starts_sub[self.i] + window_size_m * 2.0, 
                                     y_starts_sub[self.i] - window_size_m), 
                             x=slice(x_starts_sub[self.i] - window_size_m,
                                     x_starts_sub[self.i] + window_size_m * 2.0)).persist()
        shade_sub = self.shade_xr_past.sel(y=slice(y_starts_sub[self.i] + window_size_m * 2.0, 
                                     y_starts_sub[self.i] - window_size_m), 
                             x=slice(x_starts_sub[self.i] - window_size_m,
                                     x_starts_sub[self.i] + window_size_m * 2.0)).persist()
        
        tpi_sub = self.tpi_xr_past.sel(y=slice(y_starts_sub[self.i] + window_size_m * 2.0, 
                                     y_starts_sub[self.i] - window_size_m), 
                             x=slice(x_starts_sub[self.i] - window_size_m,
                                     x_starts_sub[self.i] + window_size_m * 2.0)).persist()

        poly_tmp = hv.Polygons(hv.Bounds((x_starts_sub[self.i],
                                      y_starts_sub[self.i],
                                      x_starts_sub[self.i] + window_size_m,
                                      y_starts_sub[self.i] + window_size_m))).opts('Polygons', line_color='red', fill_color=None, **self.map_opts)
        
        if len(self.burrow_poly_list[self.i]) > 0:
            #_burrows = hv.Polygons(self.burrow_poly_list[self.i])
            #tile_burrows = self.burrow_poly_list[self.i]

            #_burrows = tile_burrows.hvplot(**self.saved_poly_opts)
        
            #self.burrow_stream = streams.PolyDraw(source=_burrows, drag=True,
            #                           show_vertices=False, styles=self.draw_poly_opts)

            #self.burrow_edit_stream = streams.PolyEdit(source=_burrows, shared=True)
            _burrows = self.update_burrows()
            
            return pn.Column(
                pn.Tabs(
                    ('RGB', (rgb_sub.hvplot.rgb(x='x', y='y', bands='band',
                                                **self.map_args).opts(**self.map_opts) 
                             * poly_tmp 
                             * _burrows)), 
                    ('NDVI', (ndvi_sub.hvplot.image(x='x', y='y',
                                                    **self.map_args).opts(cmap='viridis',
                                                                          clim=(self.ndvi_range.start,
                                                                                self.ndvi_range.end),
                                                                          colorbar=False,
                                                                          **self.map_opts)
                              * poly_tmp
                              * _burrows)), 
                    ('Shade', (shade_sub.hvplot.image(x='x', y='y', 
                                                      **self.map_args).opts(cmap='gray', 
                                                                            colorbar=False,
                                                                            **self.map_opts)
                               * tpi_sub.hvplot.image(x='x', y='y',
                                                      **self.map_args).opts(cmap='turbo',
                                                                            alpha=0.25,
                                                                            clim=(-0.10, 0.40),
                                                                            colorbar=False,
                                                                            **self.map_opts) 
                               * poly_tmp
                               * burrows))), 
                hv.Table(self.burrow_poly_list[self.i]['geometry'].astype('str')))
        
        else:
            _burrows = hv.Polygons([])

            self.burrow_stream = streams.PolyDraw(source=_burrows, drag=True,
                                       show_vertices=False, styles=self.draw_poly_opts)

            self.burrow_edit_stream = streams.PolyEdit(source=_burrows, shared=True)
            return pn.Tabs(('RGB', rgb_sub.hvplot.rgb(x='x', y='y', bands='band',
                                                   **self.map_args).opts(**self.map_opts)
                                    * poly_tmp
                                    * _burrows),
                             ('NDVI', (ndvi_sub.hvplot.image(x='x', y='y',
                                                             **self.map_args).opts(cmap='viridis',
                                                                                   clim=(self.ndvi_range.start,
                                                                                         self.ndvi_range.end),
                                                                                   colorbar=False,
                                                                                   **self.map_opts)
                                       * poly_tmp
                                       * _burrows)),
                             ('Shade', (shade_sub.hvplot.image(x='x', y='y', 
                                                               **self.map_args).opts(cmap='gray',
                                                                                     colorbar=False,
                                                                                     **self.map_opts)
                                        * tpi_sub.hvplot.image(x='x', y='y',
                                                               **self.map_args).opts(cmap='turbo',
                                                                                     alpha=0.25, 
                                                                                     clim=(-0.10, 0.40),
                                                                                     colorbar=False,
                                                                                     **self.map_opts)
                                        * poly_tmp
                                        * _burrows)))
    

In [ ]:
app = TrainApp()

In [ ]:
class TrainApp2(pm.Parameterized):
       
    pasture = '5W'
    window_size_m = 15
    num_windows = 24

    select_tile = pm.Selector(objects=['tile ' + str(i+1) for i in range(num_windows)], default='tile 1')
    
    map_opts = dict(projection=crs.UTM(13), responsive=False, width=500, height=500,
                     padding=0, tools=['pan', 'box_zoom'], 
                     active_tools=['wheel_zoom'], toolbar='left')
    
    map_args = dict(crs=crs.UTM(13), rasterize=False, project=False, dynamic=True)
    
    draw_poly_opts = dict(fill_color=['red'], fill_alpha=[0.10], line_color=['red'],
             line_width=[2])  
    
    saved_poly_opts = dict(fill_color='yellow', fill_alpha=0.20, line_color='yellow', line_width=2)
    
    rgb_xr_past = rgb_xr
    ndvi_xr_past = ndvi_xr
    shade_xr_past = shade_xr
    tpi_xr_past = tpi_xr
    
    burrow_edit_list = [[] for x in range(num_windows)]
    
    burrow_poly_list = [[] for x in range(num_windows)]
    
    #burrows = hv.Polygons(burrow_poly_list[0]).opts(line_color='green')
    
    #burrow_stream = streams.PolyDraw(source=burrows, drag=True,
    #                                 show_vertices=False, styles=poly_opts)
    #burrow_edit_stream = streams.PolyEdit(source=burrows, shared=True)
    
    #btn_save = pn.widgets.Button(name='Save polygons', button_type='primary')
    
    save_action = pm.Action(lambda x: x.param.trigger('save_action'), label='Save polygons')
    
    ndvi_range = pn.widgets.RangeSlider(start=0.05, end=0.25, step=0.01)
    
    _ndvi_range = ndvi_range.param
    #test = 0
    
    def __init__(self, **params):
        super(TrainApp2, self).__init__(**params)
    
    @pm.depends('save_action', watch=True)
    def update_burrows(self):
        self.burrow_edit_list[self.i] = self.burrow_stream.element
        
        if len(self.burrow_poly_list[self.i]) > 0:
            tile_burrows_tmp = gpd.GeoDataFrame(data=self.burrow_edit_list[self.i].data)
            tile_burrows_tmp.set_geometry(tile_burrows_tmp.apply(lambda row: Polygon(zip(row['x'], row['y'])), axis=1), inplace=True)
            tile_burrows_tmp.set_crs(epsg='32613', inplace=True)
            tile_burrows_tmp = tile_burrows_tmp.drop(columns=['x', 'y'])
            self.burrow_poly_list[self.i] = self.burrow_poly_list[self.i].append(tile_burrows_tmp)
        else:
            tile_burrows_tmp = gpd.GeoDataFrame(data=self.burrow_edit_list[self.i].data)
            tile_burrows_tmp.set_geometry(tile_burrows_tmp.apply(lambda row: Polygon(zip(row['x'], row['y'])), axis=1), inplace=True)
            tile_burrows_tmp.set_crs(epsg='32613', inplace=True)
            tile_burrows_tmp = tile_burrows_tmp.drop(columns=['x', 'y'])
            self.burrow_poly_list[self.i] = tile_burrows_tmp
        #self.burrow_poly_list[self.i][0]['x'] = self.burrow_poly_list[self.i][0]['x'].astype('int')
        #self.burrow_poly_list[self.i][0]['y'] = self.burrow_poly_list[self.i][0]['y'].astype('int')
        #self.burrow_poly_list[self.i]['line_color'] = ['green' for x in self.burrow_poly_list[0]['line_color']]
    
    @pm.depends('select_tile')
    def load_image(self):
        self.i = int(self.select_tile.split(' ')[-1])
        #self.burrow_poly_list[self.i] = self.burrow_edit_stream.data
        rgb_sub = self.rgb_xr_past.sel(y=slice(y_starts_sub[self.i] + window_size_m * 2.0, 
                             y_starts_sub[self.i] - window_size_m), 
                     x=slice(x_starts_sub[self.i] - window_size_m,
                             x_starts_sub[self.i] + window_size_m * 2.0)).persist()
        ndvi_sub = self.ndvi_xr_past.sel(y=slice(y_starts_sub[self.i] + window_size_m * 2.0, 
                                     y_starts_sub[self.i] - window_size_m), 
                             x=slice(x_starts_sub[self.i] - window_size_m,
                                     x_starts_sub[self.i] + window_size_m * 2.0)).persist()
        shade_sub = self.shade_xr_past.sel(y=slice(y_starts_sub[self.i] + window_size_m * 2.0, 
                                     y_starts_sub[self.i] - window_size_m), 
                             x=slice(x_starts_sub[self.i] - window_size_m,
                                     x_starts_sub[self.i] + window_size_m * 2.0)).persist()
        
        tpi_sub = self.tpi_xr_past.sel(y=slice(y_starts_sub[self.i] + window_size_m * 2.0, 
                                     y_starts_sub[self.i] - window_size_m), 
                             x=slice(x_starts_sub[self.i] - window_size_m,
                                     x_starts_sub[self.i] + window_size_m * 2.0)).persist()

        poly_tmp = hv.Polygons(hv.Bounds((x_starts_sub[self.i],
                                      y_starts_sub[self.i],
                                      x_starts_sub[self.i] + window_size_m,
                                      y_starts_sub[self.i] + window_size_m))).opts('Polygons', line_color='red', fill_color=None, **self.map_opts)
        
        _burrows = hv.Polygons([])
        
        #self.burrow_stream = streams.PolyDraw(source=_burrows, drag=True,
         #                          show_vertices=False, styles=self.draw_poly_opts)
        
        #self.burrow_edit_stream = streams.PolyEdit(source=_burrows, shared=True)
        #tile_burrows = self.burrow_poly_list[self.i]

        #_burrows = tile_burrows.hvplot(**self.saved_poly_opts)

        burrows_annotate = hv.annotate.instance()

        burrows_layout = burrows_annotate(_burrows, annotations=['Burrow'])

        return hv.annotate.compose(rgb_sub.hvplot.rgb(x='x', y='y', bands='band',
                                               **self.map_args).opts(**self.map_opts),
                                #* poly_tmp, 
                                #* _burrows 
                                burrows_layout)
        
        """return pn.Tabs(('RGB', hv.annotate.compose(rgb_sub.hvplot.rgb(x='x', y='y', bands='band',
                                               **self.map_args).opts(**self.map_opts) 
                                * poly_tmp 
                                #* _burrows 
                                * burrows_layout)),
                         ('NDVI', hv.annotate.compose(ndvi_sub.hvplot.image(x='x', y='y',
                                                         **self.map_args).opts(cmap='viridis',
                                                                               clim=(self.ndvi_range.start,
                                                                                     self.ndvi_range.end),
                                                                               colorbar=False,
                                                                               **self.map_opts)
                                   * poly_tmp
                                   #* _burrows
                                   * burrows_layout)),
                         ('Shade', hv.annotate.compose(shade_sub.hvplot.image(x='x', y='y', 
                                                           **self.map_args).opts(cmap='gray', 
                                                                                 colorbar=False,
                                                                                 **self.map_opts)
                                    * tpi_sub.hvplot.image(x='x', y='y',
                                                           **self.map_args).opts(cmap='turbo',
                                                                                 alpha=0.25, 
                                                                                 clim=(-0.10, 0.40),
                                                                                 colorbar=False,
                                                                                 **self.map_opts) 
                                    * poly_tmp
                                    #* _burrows 
                                    * burrows_layout)))"""

    

In [ ]:
class TrainApp(pm.Parameterized):
       
    pasture = '5W'
    window_size_m = 15
    num_windows = 24

    select_tile = pm.Selector(objects=['tile_' + str(i) for i in range(num_windows)], default='tile_0')
    
    map_opts = dict(projection=crs.UTM(13), responsive=False, xaxis=None, yaxis=None, width=500, height=500,
                     padding=0, tools=['pan', 'box_zoom'], 
                     active_tools=['wheel_zoom'], toolbar='left')
    
    map_args = dict(crs=crs.UTM(13), rasterize=False, project=False, dynamic=True)
    
    draw_poly_opts = dict(fill_color=['red'], fill_alpha=[0.10], line_color=['red'],
             line_width=[2])  
    
    saved_poly_opts = dict(fill_color='yellow', fill_alpha=0.20, line_color='yellow', line_width=2)
    
    select_base = pm.Selector(objects=['RGB', 'NDVI', 'Terrain'], default='RGB')

    #rgb_xr_past = riox.open_rasterio('train_tiles/' + pasture + select_tile.value + '_rgb.tif')
    #ndvi_xr_past = riox.open_rasterio('train_tiles/' + pasture + select_tile.value + '_rgb.tif')
    #shade_xr_past = riox.open_rasterio('train_tiles/' + pasture + select_tile.value + '_rgb.tif')
    #tpi_xr_past = riox.open_rasterio('train_tiles/' + pasture + select_tile.value + '_rgb.tif')
    
    burrow_edit_list = [[] for x in range(num_windows)]
    
    burrow_poly_list = [[] for x in range(num_windows)]
    
    #burrows = hv.Polygons(burrow_poly_list[0]).opts(line_color='green')
    
    #burrow_stream = streams.PolyDraw(source=burrows, drag=True,
    #                                 show_vertices=False, styles=poly_opts)
    #burrow_edit_stream = streams.PolyEdit(source=burrows, shared=True)
    
    #btn_save = pn.widgets.Button(name='Save polygons', button_type='primary')
    
    save_action = pm.Action(lambda x: x.param.trigger('save_action'), label='Save polygons')
    
    ndvi_range = pn.widgets.RangeSlider(start=0.05, end=0.25, step=0.01)
    
    _ndvi_range = ndvi_range.param
    #test = 0
    
    def __init__(self, **params):
        super(TrainApp, self).__init__(**params)
    
    @pm.depends('save_action', watch=True)
    def load_saved_burrows(self):
        if len(self.burrow_stream.element) > 0:
            self.burrow_edit_list[self.i] = self.burrow_stream.element

            if len(self.burrow_poly_list[self.i]) > 0:
                tile_burrows_tmp = gpd.GeoDataFrame(data=self.burrow_edit_list[self.i].data)
                tile_burrows_tmp.set_geometry(tile_burrows_tmp.apply(lambda row: Polygon(zip(row['x'], row['y'])), axis=1), inplace=True)
                tile_burrows_tmp.set_crs(epsg='32613', inplace=True)
                tile_burrows_tmp = tile_burrows_tmp.drop(columns=['x', 'y'])
                self.burrow_poly_list[self.i] = self.burrow_poly_list[self.i].append(tile_burrows_tmp)
            else:
                tile_burrows_tmp = gpd.GeoDataFrame(data=self.burrow_edit_list[self.i].data)
                tile_burrows_tmp.set_geometry(tile_burrows_tmp.apply(lambda row: Polygon(zip(row['x'], row['y'])), axis=1), inplace=True)
                tile_burrows_tmp.set_crs(epsg='32613', inplace=True)
                tile_burrows_tmp = tile_burrows_tmp.drop(columns=['x', 'y'])
                self.burrow_poly_list[self.i] = tile_burrows_tmp
            return self.burrow_poly_list[self.i].hvplot(**self.saved_poly_opts)
        else:
            return hv.Polygons([])
        
        #self.burrow_poly_list[self.i][0]['x'] = self.burrow_poly_list[self.i][0]['x'].astype('int')
        #self.burrow_poly_list[self.i][0]['y'] = self.burrow_poly_list[self.i][0]['y'].astype('int')
        #self.burrow_poly_list[self.i]['line_color'] = ['green' for x in self.burrow_poly_list[0]['line_color']]
    
    @pm.depends('select_tile', 'select_base')
    def load_base(self):      
        if self.select_base == 'RGB':
            rgb = riox.open_rasterio('train_tiles/' + self.pasture + '/' + self.select_tile + '_rgb.tif')
            rgb_img = rgb.hvplot.rgb(x='x', y='y', bands='band',
                                                   **self.map_args).opts(**self.map_opts)
            return rgb_img
        elif self.select_base == 'NDVI':
            ndvi = riox.open_rasterio('train_tiles/' + self.pasture + '/' + self.select_tile + '_ndvi.tif').squeeze()
            ndvi_img = ndvi.hvplot.image(x='x', y='y',
                                                             **self.map_args).opts(cmap='viridis',
                                                                                   clim=(self.ndvi_range.start,
                                                                                         self.ndvi_range.end),
                                                                                   colorbar=False,
                                                                                   **self.map_opts)
            return ndvi_img
        elif self.select_base == 'Terrain':
            shade = riox.open_rasterio('train_tiles/' + self.pasture + '/' + self.select_tile + '_shade.tif').squeeze()
            tpi = riox.open_rasterio('train_tiles/' + self.pasture + '/' + self.select_tile + '_tpi.tif').squeeze()
            terrain_img = (shade.hvplot.image(x='x', y='y', 
                                         **self.map_args).opts(cmap='gray',
                                                               colorbar=False,
                                                               **self.map_opts) * tpi.hvplot.image(x='x', y='y',
                                                               **self.map_args).opts(cmap='turbo',
                                                                                     alpha=0.25, 
                                                                                     clim=(-0.10, 0.40),
                                                                                     colorbar=False,
                                                                                     **self.map_opts))
            return terrain_img

    
    @pm.depends('ndvi_range')
    def plot_ndvi(self):
        self.ndvi_img = self.ndvi_img.opts(clim=(self.ndvi_range.start,
                                                 self.ndvi_range.end))
    
    def create_layout(self):
        
        return pn.Row(self.load_base, self.param)
    

In [ ]:
app = TrainApp()